In [48]:
import sys
import os

# Add the project root to sys.path (not src directly!)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [49]:
import numpy as np
import pandas as pd
from src.recommender import UserBasedCF
from src.evaluation import calculate_rmse

In [50]:
user_item_matrix_ubcf = pd.read_csv(r'C:\Users\PC\Desktop\collaborative-filtering-recommender\results\user_item_matrix.csv', index_col='user_id_ml')

In [51]:
test_ratings = pd.read_csv(r'C:\Users\PC\Desktop\collaborative-filtering-recommender\results\test_ratings.csv')

In [52]:
ubcf_model = UserBasedCF(min_common_items=5)
ubcf_model.fit(user_item_matrix_ubcf)

In [53]:
def predict_ratings_ubcf(model, test_df):
    predictions = []
    for _, row in test_df.iterrows():
        pred = model.predict_rating_ubcf(row['user_id_ml'], row['item_id_ml'])
        predictions.append(pred)
    return np.array(predictions)

In [54]:
ubcf_predictions = predict_ratings_ubcf(ubcf_model, test_ratings)

In [55]:
test_ratings['predicted_rating_ubcf'] = ubcf_predictions

In [56]:
test_ratings[['user_id_ml', 'item_id_ml', 'rating_ml', 'predicted_rating_ubcf']].to_csv(
    r'C:\Users\PC\Desktop\collaborative-filtering-recommender\results\ubcf_predictions.csv', 
    index=False
)

In [ ]:
def generate_topn_ubcf(user_id, n=10):
    # Get user's rated items
    user_rated = user_item_matrix_ubcf.loc[user_id].dropna().index.astype(int)
    
    # Get all possible items
    all_items = user_item_matrix_ubcf.columns.astype(int)
    
    # Find unrated items
    unrated = [i for i in all_items if i not in user_rated]
    
    # Predict ratings for unrated items
    preds = []
    for item_id in unrated:
        pred = ubcf_model.predict_rating_ubcf(user_id, item_id)
        if not np.isnan(pred):
            preds.append((item_id, pred))
    
    # Return top N recommendations
    return sorted(preds, key=lambda x: x[1], reverse=True)[:n]

In [57]:
user_id_example = 7
top10_ubcf = generate_topn_ubcf(user_id_example)
print(f"Top 10 recommendations for user {user_id_example}:")
for item_id, rating in top10_ubcf:
    print(f"Item {item_id}: Predicted rating {rating:.2f}")

Top 10 recommendations for user 7:
Item 511: Predicted rating 4.80
Item 603: Predicted rating 4.75
Item 50: Predicted rating 4.70
Item 318: Predicted rating 4.67
Item 474: Predicted rating 4.67
Item 42: Predicted rating 4.60
Item 285: Predicted rating 4.60
Item 463: Predicted rating 4.60
Item 497: Predicted rating 4.60
Item 480: Predicted rating 4.56
